In [1]:
import h2o
import numpy as np
import pandas as pd

In [3]:
pip install h2o==3.30.0.1

Note: you may need to restart the kernel to use updated packages.


In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.3 (build 10.0.2+13, mixed mode)
  Starting server from C:\Users\ASUS\anaconda03\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ASUS\AppData\Local\Temp\tmp30m015qb
  JVM stdout: C:\Users\ASUS\AppData\Local\Temp\tmp30m015qb\h2o_ASUS_started_from_python.out
  JVM stderr: C:\Users\ASUS\AppData\Local\Temp\tmp30m015qb\h2o_ASUS_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,8 months and 26 days !!!
H2O_cluster_name:,H2O_from_python_ASUS_zgec60
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.973 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
reviews_df=pd.read_csv('reviews.txt',sep='~~')


C:\Users\ASUS\anaconda03\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


FileNotFoundError: [Errno 2] No such file or directory: 'reviews.txt'

In [1]:
import os

In [2]:
os.chdir(r"C:\Users\ASUS\Desktop\DataSet\consumers")

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
df = pd.read_csv('consumer_compliants.csv')

In [8]:
print(df.shape)
df.head(10)

(57453, 18)
  Date received                      Product  \
0      4/3/2020        Vehicle loan or lease   
1     3/12/2020              Debt collection   
2      2/6/2020        Vehicle loan or lease   
3      3/6/2020  Checking or savings account   
4     2/14/2020              Debt collection   
5      3/2/2020                     Mortgage   
6     3/15/2020              Debt collection   
7     4/28/2020  Credit card or prepaid card   
8     3/19/2020              Debt collection   
9      3/3/2020  Credit card or prepaid card   

                                  Sub-product  \
0                                        Loan   
1                            Payday loan debt   
2                                        Loan   
3                             Savings account   
4                                Medical debt   
5                      Other type of mortgage   
6                               I do not know   
7  General-purpose credit card or charge card   
8                 

In [10]:
df.Product.value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [13]:
df.Company.value_counts()

CITIBANK, N.A.                           3226
CAPITAL ONE FINANCIAL CORPORATION        2711
BANK OF AMERICA, NATIONAL ASSOCIATION    2580
JPMORGAN CHASE & CO.                     2409
WELLS FARGO & COMPANY                    2001
                                         ... 
Home Financial Corporation                  1
Credit Solutions LLC                        1
Helm Associates                             1
SOUTHPOINT FINANCIAL SERVICES, INC.         1
Collection Associates, Ltd.                 1
Name: Company, Length: 2197, dtype: int64

In [14]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [15]:
complaints = df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})

In [16]:
pd.set_option('display.max_colwidth',-1)
complaints

C:\Users\ASUS\anaconda03\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [17]:
X_train, X_hold = train_test_split(complaints, test_size=0.7, random_state=111) 

In [18]:
X_train['Product'].value_counts()

Debt collection                6582
Credit card or prepaid card    3974
Mortgage                       2802
Checking or savings account    2135
Student loan                   919 
Vehicle loan or lease          823 
Name: Product, dtype: int64

In [19]:
stemmer = PorterStemmer()

In [22]:
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text) if (len(word)>3 and len(word).strip('Xx/')>2)]
    stems = [stemmer.stem(item) for item in tokens] 
    return tokens                                              

In [24]:
vectorizer = TfidfVectorizer(tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000, use_idf=False, norm=None)
tf_vectors = vectorizer.fit_transform(X_train.complaints)

In [25]:
tf_vectors.A

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 1., ..., 0., 0., 0.],
       ...,
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.]])

In [27]:
vectorizer.get_feature_names()

['00',
 '000',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '11',
 '110',
 '1100',
 '11000',
 '12',
 '120',
 '1200',
 '12000',
 '13',
 '130',
 '1300',
 '14',
 '140',
 '1400',
 '14000',
 '15',
 '150',
 '1500',
 '15000',
 '16',
 '160',
 '1600',
 '1692',
 '1692g',
 '17',
 '170',
 '1700',
 '18',
 '180',
 '1800',
 '19',
 '190',
 '1900',
 '1st',
 '20',
 '200',
 '2000',
 '20000',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '21',
 '210',
 '2100',
 '22',
 '220',
 '2200',
 '23',
 '230',
 '2300',
 '24',
 '240',
 '2400',
 '25',
 '250',
 '2500',
 '26',
 '260',
 '2600',
 '27',
 '28',
 '280',
 '2800',
 '29',
 '2900',
 '2nd',
 '30',
 '300',
 '3000',
 '31',
 '310',
 '32',
 '320',
 '34',
 '340',
 '35',
 '350',
 '3500',
 '36',
 '360',
 '37',
 '370',
 '39',
 '390',
 '3rd',
 '40',
 '400',
 '4000',
 '420',
 '45',
 '450',
 '4500',
 '460',
 '470',
 '48',
 '49',
 '50',
 '500',
 '5000',
 '550',
 '60',
 '600',
 '6000',
 '609',
 '623',
 '650',
 '70',
 '700',
 '72',
 '75',
 '750',
 '80',
 '800

In [29]:
lda = decomposition.LatentDirichletAllocation(n_components = 6, max_iter=3, learning_method='online', learning_offset=50, n_jobs=-1, random_state=111)
w = lda.fit_transform(tf_vectors)
c = lda.components_

In [30]:
w

array([[0.57448279, 0.16839295, 0.00183818, 0.00182826, 0.0018298 ,
        0.25162801],
       [0.14448842, 0.00388946, 0.83987928, 0.00389837, 0.00392174,
        0.00392273],
       [0.3001403 , 0.13072366, 0.00113417, 0.00112689, 0.07732454,
        0.48955045],
       ...,
       [0.27298601, 0.12333825, 0.00251177, 0.00251111, 0.51423721,
        0.08441566],
       [0.92330611, 0.01542332, 0.01541013, 0.01522152, 0.01531014,
        0.01532879],
       [0.00123918, 0.12136988, 0.13568173, 0.48271413, 0.00123866,
        0.25775642]])

In [31]:
c

array([[7.57193508e+02, 2.25629091e+01, 3.27526755e+02, ...,
        4.80845550e+01, 5.43824505e+00, 8.81839809e+01],
       [1.33233983e+04, 2.27522497e+00, 2.41750582e+02, ...,
        1.21017926e+01, 1.72092938e-01, 4.88354613e+01],
       [2.23325534e+02, 1.71306202e-01, 1.22880546e+02, ...,
        2.60090158e+01, 4.05153761e+00, 1.87477227e+01],
       [5.26228394e+02, 7.02523434e+01, 2.28253707e+02, ...,
        3.28792699e+01, 2.81806527e+01, 2.19995000e+01],
       [3.46765820e+03, 2.31150199e+01, 2.45187427e+02, ...,
        1.73222811e+01, 1.17844344e+01, 1.78460110e+01],
       [2.97821225e+03, 6.56405340e+01, 3.45048594e+02, ...,
        5.99243468e-01, 4.49645137e+01, 8.76378242e+01]])

In [34]:
num_words=15

vocab = np.array(vectorizer.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in c])
topics = [' '.join(t) for t in topic_words]

In [35]:
topics

['card credit account called told did said number charges fraud phone dispute charge capital received',
 'xx 00 payment received 2019 account letter paid balance statement 2020 date sent 19 credit',
 'debt credit collection account report company information reporting letter agency sent provide original consumer law',
 'xx loan mortgage told home called company said received did phone time insurance asked sent',
 'account bank 00 money check funds xx checking wells fargo america told deposit branch chase',
 'payment credit payments loan pay late 00 paid account month time balance make years loans']

In [38]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(w, 2), columns= colnames, index= docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [39]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.57,0.17,0.00,0.00,0.00,0.25,0
Doc1,0.14,0.00,0.84,0.00,0.00,0.00,2
Doc2,0.30,0.13,0.00,0.00,0.08,0.49,5
Doc3,0.09,0.72,0.19,0.00,0.00,0.00,1
Doc4,0.00,0.23,0.35,0.42,0.00,0.00,3
...,...,...,...,...,...,...,...
Doc17230,0.63,0.09,0.00,0.00,0.27,0.00,0
Doc17231,0.25,0.17,0.17,0.05,0.27,0.08,4
Doc17232,0.27,0.12,0.00,0.00,0.51,0.08,4
Doc17233,0.92,0.02,0.02,0.02,0.02,0.02,0


In [40]:
X_train.head()

,complaints,Product,Company
17092,"I received a warning from my credit watch service that Chase has reported me to credit bureaus as being late for payment. When I checked with the credit card company Chase, I was told that I was late making payments and I was 2 months late and they closed my account and reported me. \nI do not use this card and my existing card has expired on XX/XX/2019. I have not received a new card and I haven't used this card long time. I was told by XXXX from Chase Credit Bureau Specialist that the balance is left over balance transfer fee and interest. \nI am a XXXX veteran and during these times it is unbelievable and very selfish for them to report me to agencies for approximately {$100.00}. \nOnce again I haven't received a new card when my old one was expired. I haven't received a call or text to tell me I owed money to them and I haven't received anything in writing. They bypassed any means to communicate and resolve the issue but reported me. \nWouldn't it be reasonable to ask why they did not try to reach me? \nI understand they send statements in emails but I have not received any emails with regards to this situation from them.",Credit card or prepaid card,JPMORGAN CHASE & CO.
16043,"I am a victim of identity theft. An identity thief used my personal information without my permission to open an account and make purchases with. This debt is not mine. \n\nIn accordance with the Fair Debt Collection Practices Act, I am asking you to stop collection proceedings against me and stop communicating with me about this debt, except as the Fair Credit Reporting Act allows. I also ask that you notify the business where the account was opened and tell them the debt is the result of identity theft.",Debt collection,"Hunter Warfield, Inc."
54633,"I opened an with Kay Jewelers on XX/XX/XXXX with a limit of {$750.00}. The account was paid off and closed at my request in XXXX of XXXX. On XX/XX/XXXX at approximately XXXX @ XXXX XXXX XXXX in XXXX XXXX South Carolina I went in and opened another account. The manager of that store also told me that I no longer had an account and I would have to re-apply. I reapplied and received a limit this time of {$1000.00}. I spent {$560.00}. XXXX of the {$1000.00}. The first mistake is XXXX never sent me a bill to make my first payment. The second mistake is that I went into the XXXX @ XXXX XXXX in XXXX XXXX Maryland on XX/XX/XXXX and bought {$200.00} dollars worth of jewelry and then got a bill from two different credit card company saying that they were kay jewelers which was XXXX XXXX, and XXXX XXXX XXXX. When calling both companies, I was told that I had to pay them both, but it was impossible because I used the XXXX account # that XXXX XXXX gave. So I disputed, there was no previous XXXX account on my credit report and according to XXXX XXXX, all in store cards were sold, and didn't report and mine was a in store card and a account that had been paid off and closed out before. As soon as I made noise and disputed with both companies, no one knew what I was speaking account so I reported both accounts as fraud. I have no problem paying what I owe but will only pay what I owe. A supervisor from kay jewelers called and let me know that they would start and investigation in to the account andn let me know the account but in the mean time, halt everything, so I haled payments. The other company told me the same thing. XXXX swears this account was opened i XXXX of XXXX, when it was opened in XXXX of XXXX, they are reporting 10 missed payments, and XXXX is reporting 5 missed payments, I never missed anything becuase no one could figure out what was going on, it was XXXX XXXX issue putting me on 2 different accounts, not mine, This issue went on for months, and we finallly resolved it with XXXX XXXX and paid what was past due. But instead of taking the lates off because of the fraud issue as they prmised, they put more on there and decreased my limit to XXXX. My credit is importan

In [44]:
whold = lda.transform(vectorizer.transform(X_hold.complaints[:5]))

In [45]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.complaints[:5]))]
df_doc_topic = pd.DataFrame(np.round(whold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [46]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.00,0.13,0.00,0.00,0.84,0.02,4
Doc1,0.00,0.00,0.18,0.72,0.09,0.00,3
Doc2,0.67,0.01,0.01,0.01,0.01,0.31,0
Doc3,0.00,0.00,0.00,0.34,0.00,0.65,5
Doc4,0.24,0.17,0.00,0.00,0.00,0.58,5


In [47]:
X_hold.head()

,complaints,Product,Company
30060,"I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX/XX/2019. In the intervening time, my business account became overdrawn. I had the money in my personal account to bring it back into the black, but BB & T could not make the transfer until I went into a branch. During this time, I incurred an astonishing {$320.00} in overdraft fees in my business account because I had no way of transferring money between the two accounts. \n\nWhen I went into the branch, I met with XXXX XXXX in XXXX XXXX County, Florida. She investigated the accounts and told me she would link them and that I would have online access in XXXX hours. The online access still never material, I was not able to transfer money and I received an additional {$36.00} overdraft fee. This brought the total to {$360.00} in overdraft fees. \n\nI had only opened the second account on BB & Ts advice ; I could have had my customer send a bank wire directly into business account in the first week of XXXX to keep the account positive. My TOTAL deposits between the two accounts was always positive. \n\nI requested a refund and was told they would not refund any of the fees.",Checking or savings account,BB&T CORPORATION
53473,"To who it may concern, My concern is regarding Shellpoint Mortgage Servicing Company. This company has received my monthly mortgage payments and have failed to account it as payments received. My bank has sent me proof the account was cleared and paid. This company is trying to foreclosed on my house and has sent me a letter indicating they sent my file to their Loss Mitigation dept. The person who they assigned it to is XXXX XXXX XXXX x XXXX. Their corporate number is XXXX. \n\nI have can send proof of my claim from above should you need it. Please assist.",Mortgage,"Shellpoint Partners, LLC"
35879,I contacted XXXX about fraudulent charges that were made on my account and the customer service representative told me that they wouldnt be able to issue anew card or remove my fraudulent charges since the account was closed due to non-payment. I was unaware of this these charging that were made.. To my knowledge I didnt owe a payment because the account wasnt being used.,Credit card or prepaid card,"EQUIFAX, INC."
20993,"I first applied for the Fedloan Serving program in XXXX in hopes of getting loan forgiveness since I've been paying on my student loans since late XXXX or early XXXX and I have been a public servant since XXXX, XXXX. I have never missed a payment to any of the various loan companies that owned my loans. The Fedloan Program turned me down in XXXX, saying my loans, which were originally Federal Stafford Loans, did not qualify me for the Program. In XXXX, XXXX, I was solicited by XXXX XXXX and told I could get into this program to reduce my loan payments. I was charged {$690.00} by XXXX XXXX to get me into the Fedloan Serving program. My loan payments went down to $ XXXX/month for the first year. In XXXX, XXXX, Fedloan Servicing informed me via email that I needed to recertify my "" income-drive